In [ ]:
#import libraries
import tweepy
import pandas as pd
import numpy as np
from textblob import TextBlob
from wordcloud import WordCloud, STOPWORDS
from PIL import Image
import matplotlib.pyplot as plt
import re

from collections import Counter

In [ ]:
#1Data Collection

In [ ]:
#Twitter API credentials
api_key = 'XXXXXXXXXXXXXXXXXXXXXXX'
api_key_secret = 'XXXXXXXXXXXXXXXXXXXXXXX'
access_token = 'XXXXXXXXXXXXXXXXXXXXXXX'
access_token_secret = 'XXXXXXXXXXXXXXXXXXXXXXX'

In [ ]:
#create the authentication object
auth = tweepy.OAuthHandler(api_key, api_key_secret)

#set the accesstoken and accesstokensecret
auth.set_access_token(access_token, access_token_secret)

#create the API object while passing in the auth information
api = tweepy.API(auth, wait_on_rate_limit=True) #sleeps when API limit is reached
sleep_on_rate_limit=False

In [ ]:
#keywords and hashtags to look out for
keywords = "#BlackPanther2 OR #WakandaForever OR #BlackPanther"
search_query = keywords + " -filter:retweets AND -filter:replies" #filters out retweets and replies

In [ ]:
#create a function to get the tweets and append it to data
data = []

def get_tweets(search_query, limit):
    tweets = tweepy.Cursor(api.search_tweets, q=search_query, count = 100, lang = 'en', tweet_mode= 'extended').items(limit)
    for tweet in tweets:
        data.append([tweet.id, #every tweet has a unique id, so get the tweet id, we will use it to check for duplicates
        tweet.user.screen_name, #to get the username of the "tweeter"
        tweet.full_text,        #to get the tweet
        tweet.favorite_count,   #to get the number of likes on the tweet
        tweet.retweet_count,    #the number of retweets on the tweet
        tweet.user.location,    #to get the location of the user
        tweet.created_at])      #date and time of tweet
        #print(tweet)
        
#  Pass in search query and the number of tweets to retrieve
get_tweets(search_query, 100000)  

#Assign names to each column and print the df
df1 = pd.DataFrame(data, columns = ['Id','Username', 'Tweet', 'No. of Likes', 'No. of Retweets', 'Location', 'Date'])
print(df1.tail())  #print the tail of the df to get the id of the last tweet, we will use the last id to get older tweets

In [ ]:
#get older tweets with max_id
data = []

def get_tweets2(search_query, limit):
    tweets = tweepy.Cursor(api.search_tweets, q=search_query,max_id = XXXXXXXXXX, count = 100, lang = 'en', tweet_mode= 'extended').items(limit)
    for tweet in tweets:
        data.append([tweet.id,
        tweet.user.screen_name,
        tweet.full_text,
        tweet.favorite_count, 
        tweet.retweet_count,
        tweet.user.location,
        tweet.created_at])
        #print(tweet)
        
#  Pass in search query and the number of tweets to retrieve
get_tweets2(search_query, 50000)  

#Assign names to each column and print the df
df2 = pd.DataFrame(data, columns = ['Id','Username', 'Tweet', 'No. of Likes', 'No. of Retweets', 'Location', 'Date'])
print(df2)

In [ ]:
#combine the dataframes with concat
df = pd.concat([df1, df2], ignore_index=True)
df

In [ ]:
#2 Data Extraction

In [ ]:
# Define a function to extract hashtags and remove # with re
def getHashtags(tweet):
    tweet = tweet.lower()  #to reduce the tweets to lowercase
    tweet = re.findall(r'\#\w+',tweet) #find all words with # at the start, ie; hashtags
    return " ".join(tweet)

#apply the function to the Tweet column to extract the hashtags
df['Hashtags'] = df['Tweet'].apply(getHashtags)
df.head()

hashtags_list = df['Hashtags'].tolist() #to convert the data elements into a list

# Iterate over all hashtags and split where there is more than one hashtag
hashtags = []
for item in hashtags_list:
    item = item.split()
    for i in item:
        hashtags.append(i)

# Determine count of all hashtags used
counts = Counter(hashtags)
hashtags_df = pd.DataFrame.from_dict(counts, orient='index').reset_index()
hashtags_df.columns = ['Hashtags', 'Count']
hashtags_df.sort_values(by='Count', ascending=False, inplace=True)

#print hashtags_df
hashtags_df

In [ ]:
#Wakanda_cast = ["Shuri","Nakia","Okoye","M'baku","Ayo","Ironheart","Aneka","Namor","Everet","T'Challa","Ramonda"]

In [ ]:
#I created a group for the first names of the cast and their respective character names. Noticed that surnames aren't there? It'd to avoid duplicates
Wakanda_cast = "Chadwick|Tenoch|Letitia|Lupita|Angela|Winston|Mbaku|Shuri|Nakia|Okoye|Danai|M'baku|Ayo|Ironheart|Aneka|Namor|Everet|T'Challa|Ramonda|Namora|W'Kabi|Wkabi|Mabel|Princess|Queen"
# Define function to extract casts from each Tweet
def getCast(tweet):
    tweet = tweet.lower()  
    tweet = re.findall(Wakanda_cast, tweet, flags=re.IGNORECASE)
    return " ".join(tweet)

#Extract casts to a new column
df['Movie_cast'] = df['Tweet'].apply(getCast)
df

In [ ]:
#create a function to replace their actual names with the character names
def castNames(wakanda_cast):
    replacements = [("Letitia", "shuri"),
                    ("Princess", "shuri"),
                    ("Lupita", "nakia"),
                    ("Danai", "okoye"),
                    ("Winston", "m'baku"),
                    ("Mbaku", "m'baku"),
                    ("Florence", "ayo"), 
                    ("Dominique", "ironheart"), 
                    ("Michaela", "aneka"),
                    ("Mabel", "namora"),
                    ("Tenoch", "namor"), 
                    ("Daniel", "w'kabi"),
                    ("Wkabi", "w'kabi"),
                    ("Martin", "everet"), 
                    ("Chadwick", "t'challa"), 
                    ("Angela", "ramonda"),
                    ("Queen", "ramonda")]
    for act,rep in replacements:
        wakanda_cast = re.sub(act, rep, wakanda_cast, flags=re.IGNORECASE)
    return wakanda_cast

df['Movie_cast'] = df['Movie_cast'].apply(castNames)
df

In [ ]:
#convert the df to a list
cast_list = df['Movie_cast'].tolist()

# Iterate over all cast names and split where there is more than one cast
cast = []
for item in cast_list:
    item = item.split()
    for i in item:
        cast.append(i)

# Determine count of all cast
counts = Counter(cast)
cast_df = pd.DataFrame.from_dict(counts, orient='index').reset_index()
cast_df.columns = ['Cast', 'Count']
cast_df.sort_values(by='Count', ascending=False, inplace=True)

cast_df

In [ ]:
#3 Data Preprocessing

In [ ]:
#create a function to clean the tweets, ie; get rid of @, RT, numbers, url and so on
def cleanTxt(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text) #to remove @ mentions, + means 1 or more
    text = re.sub(r'[#:_\!/]', '', text) #to remove symbols
    text = re.sub(r'RT[\s]+', '', text) #to remove RT mentions. [\s] means 1 or more white spaces
    text = re.sub(r'https?:\/\/\+S', '', text) #remove hyperlinks. s? means may or may not have an s. +S = may have whitespaces 
    text = re.sub(r'httpst', '', text)
    
    return text

#apply function to the Tweet column
df['Tweet'] = df['Tweet'].apply(cleanTxt)

df

In [ ]:
#4 data cleaning

In [ ]:
#check for missing values
df.isnull().sum()

In [ ]:
#Fill missing locations with "Unspecified"
df["Location"]=df["Location"].fillna('Unspecified')
df

In [ ]:
#This is to check if the missing locations have been replaced
df['Location'].isnull().sum()

In [ ]:
#check for duplicates
df.duplicated(subset='Id').sum()

In [ ]:
#drop duplicates in the dataframe
df.drop_duplicates(subset = 'Id',inplace=True)
df

In [ ]:
#drop unnecessary column(s). Here, I'm dropping id, since I have confirmed there are no duplicates
df = df.drop(['Id'], axis=1)
df

In [ ]:
#5 Sentiments Analysis

In [ ]:
#create a function to get the subjectivity
def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity

#Create a function to get the polarity
def getPolarity(text):
    return TextBlob(text).sentiment.polarity

#Create two new columns
df['Subjectivity'] = df['Tweet'].apply(getSubjectivity)
df['Polarity'] = df['Tweet'].apply(getPolarity)

#Show the new dataframe with the new columns
df

In [ ]:
#Create a function to compute the polarity(negative, neutral, positive) sentiments
def getSentiments(score):
    if score < 0:
        return 'Negative'
    elif score == 0:
        return 'Neutral'
    else:
        return 'Positive'
    
#create a new column to save the sentiments
df['Sentiments'] = df['Polarity'].apply(getSentiments)

#Show the dataframe
df

In [ ]:
#6 Wordcloud

In [ ]:
#plot the wordcloud
mask = np.array(Image.open("Images/wakanda.png"))

#Instantiate the wordcloud using
allwords = ' '.join(df['Tweet'])
wordcloud = WordCloud(
    stopwords=STOPWORDS,
    background_color="black",
    height = 2000,
    width = 3000,
    mask = mask,
    colormap = 'Blues',
    collocations = False,
    max_font_size = 200).generate(allwords)

# create wakanda forever image
plt.figure(figsize=[15,10], facecolor='none')
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.title('Word Cloud for Black Panther 2')
plt.savefig("blackpanther.png", format="png")
plt.show()

In [ ]:
#export the dataframes
df.to_csv('Black_Panther.csv')

cast_df.to_csv('cast_df.csv')

hashtags_df.to_csv('hashtags_df.csv')

In [ ]:
#7. Data Visualization
#I imported the files into power bi for the visualization